In [65]:
import os

In [66]:
%pwd

'e:\\Preparation_2024\\Syallabus\\NLP\\Text-Summarization-NLP-Project'

In [67]:
os.chdir(r"e:\Preparation_2024\Syallabus\NLP\Text-Summarization-NLP-Project")

In [68]:
%pwd

'e:\\Preparation_2024\\Syallabus\\NLP\\Text-Summarization-NLP-Project'

In [69]:
from dataclasses import dataclass
import os
from pathlib import Path

In [70]:
@dataclass(frozen=True)
class Modeltrainerconfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_eval_batch_size: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumalation_steps: int

In [71]:
from Text_summarization.constants import *
from Text_summarization.utils.common import read_yaml,create_directories    

In [72]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_modeltrainer_config(self) -> Modeltrainerconfig:
        config = self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])
        
        model_trainer_config = Modeltrainerconfig(
                root_dir= config.root_dir,
                data_path= config.data_path,
                model_ckpt= config.model_ckpt,
                num_train_epochs= params.num_train_epochs,
                warmup_steps= params.warmup_steps,
                per_device_eval_batch_size= params.per_device_eval_batch_size,
                weight_decay= params.weight_decay,
                logging_steps= params.logging_steps,
                evaluation_strategy= params.evaluation_strategy,
                eval_steps= params.eval_steps,
                save_steps= params.save_steps,
                gradient_accumalation_steps=params.gradient_accumalation_steps,
                per_device_train_batch_size=params.per_device_train_batch_size
        )

        return model_trainer_config

In [73]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [74]:
class ModelTrainer:
    def __init__(self, config: Modeltrainerconfig):
        self.config = config

   
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print(dataset_samsum_pt)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumalation_steps=self.config.gradient_accumalation_steps
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],  #Due to high size of train data i am running on test data
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()


        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        

In [75]:
try:
    config=ConfigurationManager()
    model_trainerconfig=config.get_modeltrainer_config()
    model_trainerconfig=ModelTrainer(config=model_trainerconfig)
    model_trainerconfig.train()
except Exception as e:
    raise e

[2024-05-18 16:57:56,442: INFO: common: Yaml file :config\config.yaml loaded successfully]
[2024-05-18 16:57:56,501: INFO: common: Yaml file :params.yaml loaded successfully]
[2024-05-18 16:57:56,516: INFO: common: created directory at : artifacts]
[2024-05-18 16:57:56,524: INFO: common: created directory at : artifacts/model_trainer]
